Εκφώνηση Ερωτήματος 2
******
Το εργοστάσιο ῾῾Ουράνιο Τόξο᾿᾿ έχει αναλάβει επίσης τον χρωματισμό τριών ταπετσαριών με τις κωδικές
ονομασίες: Ταπ1, Ταπ2 και Ταπ3. Η Ταπ1 έχει μπλε φόντο και κίτρινα σχέδια, η Ταπ2 έχει πράσινο
φόντο και μπλε και κίτρινα σχέδια και η Ταπ3 έχει κίτρινο φόντο και μπλε και πράσινα σχέδια. Για την
επίτευξη των παραπάνω χρωματισμών, το εργοστάσιο έχει τρία διαφορετικά μηχανήματα που το κάθε
ένα μπορεί να χρώματίσει με ένα μόνο από τα παραπάνω χρώματα (πράσινο, μπλε ή κίτρινό). Η Ταπ1
για να παραχθεί θα πρέπει πρώτα να δημιουργηθεί το μπλε φόντο και στην συνέχεια να δημιουργηθούν
τα κίτρινα σχέδια. Η Ταπ2 για να παραχθεί θα πρέπει πρώτα να δημιουργηθεί το πράσινο φόντο, στην
συνέχεια να δημιουργηθούν τα μπλε σχέδια και στο τέλος να δημιουργηθούν τα κίτρινα σχέδια. Η Ταπ3
για να παραχθεί θα πρέπει πρώτα να δημιουργηθεί το κίτρινο φόντο, στην συνέχεια να δημιουργηθούν
τα μπλε σχέδια και στο τέλος να δημιουργηθούν τα πράσινα σχέδια.
Ο χρόνος εφαρμογής κάθε χρωμάτος για την παραγωγή του κάθε τύπου ταπετσαρίας διαφέρει βάσει
του υλικού της επιφάνεις που πρέπει να χρωματιστεί. Συγκεκριμένα, αυτός ο χρόνος (σε λεπτά) δίνεται
στον παρακάτω πίνακα.

Γνωρίζοντας ότι κάθε μηχάνημα μπορεί να επεξεργαστεί μία ταπετσαρία κάθε φορά και ότι η κάθε
ταπετσαρία δεν μπορεί να επεξεργαστεί παράλληλα από διαφορετικά μηχανήματα, η διοίκηση σας ζητά να βρείτε εκείνον τον προγραμματισμό που θα αποπερατώσει την παραγωγή και των τριών τύπων
ταπετσαρίας στο μικρότερο δυνατό χρόνο.


In [1]:
import pandas as pd

import gurobipy

from pyomo.environ import *
from pyomo.gdp import *

Φορτώνω τον πίνακα με τους χρόνους και την απαιτούμενη προετοιμασία για κάθε ταπετσαρία.

In [2]:
TASKS = {
    ('Ταπ_1','Μπλε')   : {'dur': 45, 'prec': None},
    ('Ταπ_1','Κίτρινο') : {'dur': 10, 'prec': ('Ταπ_1','Μπλε')},
    ('Ταπ_2','Μπλε')   : {'dur': 20, 'prec': ('Ταπ_2','Πράσινο')},
    ('Ταπ_2','Πράσινο')  : {'dur': 10, 'prec': None},
    ('Ταπ_2','Κίτρινο') : {'dur': 34, 'prec': ('Ταπ_2','Μπλε')},
    ('Ταπ_3','Μπλε')   : {'dur': 12, 'prec': ('Ταπ_3','Κίτρινο')},
    ('Ταπ_3','Πράσινο')  : {'dur': 17, 'prec': ('Ταπ_3','Μπλε')},
    ('Ταπ_3','Κίτρινο') : {'dur': 28, 'prec': None},   
}

Σκοπός μας είναι να ελαχιστοποήσουμε τον συνολικό χρόνο που απαιτείται για την κάθε ταπετσαρία/εργασία, δηλαδή minimize finish = start_time + duration. 

Για τελειώσει, πρέπει να έχουν τελείωσει όλες οι διαδικασίες από όλες τις εργασίες/ταπετσαρίες, δηλαδή θέλουμε η χρονινκή στιγμή finish να είναι μεγαλύτερη από τους χρόνους τελειώνουν οι τελευταίες διαδικασίες για κάθε εργασία:

finish ≥ start13 + DUR13 
finish ≥ start23 + DUR23 
finish ≥ start32 + DUR32 

Π.χ. για την εργασία 1 θέλουμε να μπεί πρώτα το μηχάνημα 1 και μετά το μηχάνημα 3. Άρα πρέπει η διαδικασία στο μηχάνημα 1 να τελείωσει όταν ξεκινάει η διαδικασία στο μηχάνημα 3. 

start11 + DUR11 ≤ start13

Ομοίως, για την εργασία 2 πρέπει να ξεκινήσει με το μηχάνημα 2, μετά να πάει να πάει στο 1 και τέλος στο 3. Άρα θέλουμε:

start22 + DUR22 ≤ start21 
start21 + DUR21 ≤ start23 

Για την εργασία 3, έχουμε την σειρά 3, 1 και 2. Άρα θέλουμε:

start33 + DUR33 ≤ start31 
start31 + DUR31 ≤ start32 

********
Συμβολισμός: start23 == ο χρόνος που η εργασία 2 μπαίνει στο μηχάνημα 3 (2,3).


In [3]:
def jobshop_model(TASKS):
    
    model = ConcreteModel()

    model.TASKS = Set(initialize = TASKS.keys(), dimen=2)
    model.JOBS = Set(initialize = list(set([j for (j,m) in model.TASKS])))
    model.MACHINES = Set(initialize = list(set([m for (j,m) in model.TASKS])))
    
    model.TASKORDER = Set(initialize = model.TASKS * model.TASKS, dimen=4, 
        filter = lambda model, j, m, k, n: (k,n) == TASKS[(j,m)]['prec'])
    
    model.DISJUNCTIONS = Set(initialize = model.JOBS * model.JOBS * model.MACHINES, dimen=3,
        filter = lambda model, j, k, m: j < k and (j,m) in model.TASKS and (k,m) in model.TASKS)
    
    model.dur = Param(model.TASKS, initialize=lambda model, j, m: TASKS[(j,m)]['dur'])

    ub = sum([model.dur[j, m] for (j,m) in model.TASKS])
    
    model.makespan = Var(bounds=(0, ub))
    model.start = Var(model.TASKS, bounds=(0, ub))
    
    model.objective = Objective(expr = model.makespan, sense = minimize)

    model.finish = Constraint(model.TASKS, rule=lambda model, j, m:  
        model.start[j,m] + model.dur[j,m] <= model.makespan)
    
    model.preceding = Constraint(model.TASKORDER, rule=lambda model, j, m, k, n: 
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])
    
    model.disjunctions = Disjunction(model.DISJUNCTIONS, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m], 
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])
    
    TransformationFactory('gdp.hull').apply_to(model)
    return model

jobshop_model(TASKS)

TASKS*TASKS


Χρησιμοποιώ το SolverFactory('gurobi') και ορίζω:

    Job: η ταπετσαρία που πρέπει να φτιαχτεί (εργασία)

    Machine: το μηχάνημα για κάθε χρώμα (1, 2, 3)

    Start: η στιγμή που ξεκινάει η αντίστοιχη διαδικασία 

    Duration: η διάρκεια μέχρι να ολοκληρωθεί η διαδικασία
    
    Finish: η στιγμή που τελειώνεια η αντίστοιχη διαδικασία

In [4]:
def jobshop_solve(model):
    SolverFactory('gurobi').solve(model)
    
    results = [{'Job': j,
                'Machine': m,
                'Start': model.start[j, m](), 
                'Duration': model.dur[j,m], 
                'Finish': model.start[(j, m)]() + model.dur[j,m]}
               for j,m in model.TASKS]
    return results

def jobshop(TASKS):
    return jobshop_solve(jobshop_model(TASKS))

results = jobshop(TASKS)
results

TASKS*TASKS


[{'Job': 'Ταπ_1',
  'Machine': 'Μπλε',
  'Start': 42.0,
  'Duration': 45,
  'Finish': 87.0},
 {'Job': 'Ταπ_1',
  'Machine': 'Κίτρινο',
  'Start': 87.0,
  'Duration': 10,
  'Finish': 97.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Μπλε',
  'Start': 10.0,
  'Duration': 20,
  'Finish': 30.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Πράσινο',
  'Start': 0.0,
  'Duration': 10,
  'Finish': 10.0},
 {'Job': 'Ταπ_2',
  'Machine': 'Κίτρινο',
  'Start': 30.0,
  'Duration': 34,
  'Finish': 64.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Μπλε',
  'Start': 30.0,
  'Duration': 12,
  'Finish': 42.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Πράσινο',
  'Start': 42.0,
  'Duration': 17,
  'Finish': 59.0},
 {'Job': 'Ταπ_3',
  'Machine': 'Κίτρινο',
  'Start': 0.0,
  'Duration': 28,
  'Finish': 28.0}]

In [5]:
schedule = pd.DataFrame(results)

print('\nSchedule by Job')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))

print('\nSchedule by Machine')
print(schedule.sort_values(by=['Machine','Start']).set_index(['Machine', 'Job']))


Schedule by Job
               Start  Duration  Finish
Job   Machine                         
Ταπ_1 Μπλε      42.0        45    87.0
      Κίτρινο   87.0        10    97.0
Ταπ_2 Πράσινο    0.0        10    10.0
      Μπλε      10.0        20    30.0
      Κίτρινο   30.0        34    64.0
Ταπ_3 Κίτρινο    0.0        28    28.0
      Μπλε      30.0        12    42.0
      Πράσινο   42.0        17    59.0

Schedule by Machine
               Start  Duration  Finish
Machine Job                           
Κίτρινο Ταπ_3    0.0        28    28.0
        Ταπ_2   30.0        34    64.0
        Ταπ_1   87.0        10    97.0
Μπλε    Ταπ_2   10.0        20    30.0
        Ταπ_3   30.0        12    42.0
        Ταπ_1   42.0        45    87.0
Πράσινο Ταπ_2    0.0        10    10.0
        Ταπ_3   42.0        17    59.0
